In [0]:
from tensorflow.lite.python.interpreter import Interpreter
import numpy as np
import argparse
import os
import cv2

parser = argparse.ArgumentParser(description='Image Classification')
parser.add_argument('--model_path', type=str, help='Specify the model path', required=True)
parser.add_argument('--label_path', type=str, help='Specify the label map', required=True)
parser.add_argument('--top_k', type=int, help='How many top results', default=2)

args = parser.parse_args()

model_path = args.model_path 
label_path = args.label_path 
top_k_results = args.top_k

with open(label_path, 'r') as f:
    labels = list(map(str.strip, f.readlines()))


In [0]:
interpreter = Interpreter(model_path=model_path)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

input_shape = input_details[0]['shape']
size = input_shape[:2] if len(input_shape) == 3 else input_shape[1:3]

# Read image
IMAGE_NAME = 'person.png'
#IMAGE_NAME = 'no_person.png'
CWD_PATH = os.getcwd()
PATH_TO_IMAGE = os.path.join(CWD_PATH,IMAGE_NAME)
img = cv2.imread(PATH_TO_IMAGE)
image_resized = cv2.resize(img, (224, 224))
img = np.array(image_resized)
input_data = np.expand_dims(img, axis=0)

# Point the data to be used for testing and run the interpreter
interpreter.set_tensor(input_details[0]['index'], input_data)
interpreter.invoke()

# Obtain results and map them to the classes
predictions = interpreter.get_tensor(output_details[0]['index'])[0]
print(predictions)
# Get indices of the top k results
top_k_indices = np.argsort(predictions)[::-1][:top_k_results]

for i in range(top_k_results):
    print(labels[top_k_indices[i]], predictions[top_k_indices[i]] / 255.0)